# Coordination Engine Integration for Self-Healing Platform

## Overview
This notebook demonstrates how to integrate trained anomaly detection models with the Self-Healing Platform's coordination engine. It shows the complete workflow from anomaly detection to automated remediation actions.

## Prerequisites
- Trained anomaly detection models (from 02-anomaly-detection notebooks)
- Running coordination engine in the cluster
- Access to OpenShift API and Prometheus metrics

## Expected Outcomes
- Understand coordination engine API and integration patterns
- Implement real-time anomaly detection pipeline
- Demonstrate automated remediation workflows
- Test end-to-end self-healing scenarios

## References
- ADR-002: Hybrid Deterministic-AI Self-Healing Approach
- ADR-009: Bootstrap Deployment Automation Architecture
- ADR-012: Notebook Architecture for End-to-End Workflows

## Setup and Configuration

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import json
import time
import joblib
import warnings
warnings.filterwarnings('ignore')

# Kubernetes client
try:
    from kubernetes import client, config
    k8s_available = True
    print("✅ Kubernetes client available")
except ImportError:
    k8s_available = False
    print("⚠️ Kubernetes client not available - using simulation mode")

# Try to import common functions, with fallback
try:
    from common_functions import (
        setup_environment, print_environment_info,
        generate_synthetic_timeseries, validate_data_quality,
        load_processed_data
    )
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Using fallback implementations")
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}
    def print_environment_info(env_info):
        print(f"📁 Data dir: {env_info.get('data_dir', 'N/A')}")
    def generate_synthetic_timeseries(*args, **kwargs):
        return pd.DataFrame({'value': np.random.random(100)})
    def validate_data_quality(df, name=''):
        return {'valid': True}
    def load_processed_data(filename):
        return None

print("✅ Libraries imported successfully")

In [ ]:
# Set up environment
env_info = setup_environment()
print_environment_info(env_info)

# Coordination Engine Configuration
COORDINATION_ENGINE_CONFIG = {
    'base_url': 'http://coordination-engine.self-healing-platform.svc.cluster.local:8080',
    'timeout': 30,
    'retry_attempts': 3,
    'retry_delay': 5
}

# Self-Healing Actions Configuration
REMEDIATION_ACTIONS = {
    'pod_restart': {
        'description': 'Restart problematic pods',
        'severity_threshold': 0.7,
        'cooldown_minutes': 10
    },
    'scale_up': {
        'description': 'Scale up deployment replicas',
        'severity_threshold': 0.8,
        'cooldown_minutes': 15
    },
    'resource_adjustment': {
        'description': 'Adjust resource limits/requests',
        'severity_threshold': 0.6,
        'cooldown_minutes': 30
    },
    'alert_escalation': {
        'description': 'Escalate to human operators',
        'severity_threshold': 0.9,
        'cooldown_minutes': 5
    }
}

print(f"🎯 Coordination engine: {COORDINATION_ENGINE_CONFIG['base_url']}")
print(f"🔧 Available remediation actions: {len(REMEDIATION_ACTIONS)}")

## Coordination Engine Client

Implement client for communicating with the coordination engine.

In [ ]:
class CoordinationEngineClient:
    """
    Client for interacting with the Self-Healing Platform coordination engine
    """
    
    def __init__(self, base_url, timeout=30):
        self.base_url = base_url.rstrip('/')
        self.timeout = timeout
        self.session = requests.Session()
        
    def health_check(self):
        """
        Check if coordination engine is healthy
        """
        try:
            response = self.session.get(
                f"{self.base_url}/health",
                timeout=self.timeout
            )
            return response.status_code == 200
        except Exception as e:
            print(f"❌ Health check failed: {e}")
            return False
    
    def submit_anomaly(self, anomaly_data):
        """
        Submit anomaly detection results to coordination engine
        """
        try:
            response = self.session.post(
                f"{self.base_url}/api/v1/anomalies",
                json=anomaly_data,
                timeout=self.timeout
            )
            return response.status_code == 200, response.json() if response.status_code == 200 else None
        except Exception as e:
            print(f"❌ Failed to submit anomaly: {e}")
            return False, None
    
    def get_remediation_status(self, remediation_id):
        """
        Get status of a remediation action
        """
        try:
            response = self.session.get(
                f"{self.base_url}/api/v1/remediations/{remediation_id}",
                timeout=self.timeout
            )
            return response.status_code == 200, response.json() if response.status_code == 200 else None
        except Exception as e:
            print(f"❌ Failed to get remediation status: {e}")
            return False, None
    
    def get_metrics(self):
        """
        Get coordination engine metrics
        """
        try:
            response = self.session.get(
                f"{self.base_url}/metrics",
                timeout=self.timeout
            )
            return response.status_code == 200, response.text if response.status_code == 200 else None
        except Exception as e:
            print(f"❌ Failed to get metrics: {e}")
            return False, None

# Initialize coordination engine client
coord_client = CoordinationEngineClient(
    base_url=COORDINATION_ENGINE_CONFIG['base_url'],
    timeout=COORDINATION_ENGINE_CONFIG['timeout']
)

# Test connection
print("🔍 Testing coordination engine connection...")
if coord_client.health_check():
    print("✅ Coordination engine is healthy")
else:
    print("⚠️ Coordination engine not available - using simulation mode")
    
print("✅ Coordination engine client initialized")

## Anomaly Detection Pipeline

Load trained models and create real-time anomaly detection pipeline.

In [ ]:
class AnomalyDetectionPipeline:
    """
    Real-time anomaly detection pipeline for self-healing platform
    """
    
    def __init__(self, model_path=None):
        self.model = None
        self.scaler = None
        self.feature_columns = None
        self.last_prediction_time = None
        
        if model_path and os.path.exists(model_path):
            self.load_model(model_path)
        else:
            print("⚠️ No model provided - will create demo model")
            self._create_demo_model()
    
    def _create_demo_model(self):
        """
        Create a demo model for demonstration purposes
        """
        from sklearn.ensemble import IsolationForest
        from sklearn.preprocessing import RobustScaler
        
        print("🔧 Creating demo anomaly detection model...")
        
        # Generate sample training data
        np.random.seed(42)
        n_samples = 1000
        n_features = 10
        
        # Normal data
        X_normal = np.random.normal(0, 1, (int(n_samples * 0.9), n_features))
        # Anomalous data
        X_anomaly = np.random.normal(3, 1, (int(n_samples * 0.1), n_features))
        
        X_train = np.vstack([X_normal, X_anomaly])
        
        # Train scaler and model
        self.scaler = RobustScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        self.model = IsolationForest(
            contamination=0.1,
            n_estimators=100,
            random_state=42
        )
        self.model.fit(X_train_scaled)
        
        # Set feature columns
        self.feature_columns = [f'feature_{i}' for i in range(n_features)]
        
        print("✅ Demo model created successfully")
    
    def load_model(self, model_path):
        """
        Load trained model from disk
        """
        try:
            model_data = joblib.load(model_path)
            self.model = model_data['model']
            self.scaler = model_data['scaler']
            self.feature_columns = model_data['feature_columns']
            print(f"✅ Model loaded from {model_path}")
        except Exception as e:
            print(f"❌ Failed to load model: {e}")
            self._create_demo_model()
    
    def predict_anomaly(self, features):
        """
        Predict anomalies in feature data
        """
        if self.model is None:
            raise ValueError("No model available for prediction")
        
        # Ensure features is 2D
        if len(features.shape) == 1:
            features = features.reshape(1, -1)
        
        # Scale features
        features_scaled = self.scaler.transform(features)
        
        # Make predictions
        predictions = self.model.predict(features_scaled)
        scores = self.model.decision_function(features_scaled)
        
        # Convert to anomaly probabilities
        anomaly_mask = predictions == -1
        
        results = {
            'is_anomaly': anomaly_mask,
            'anomaly_scores': scores,
            'severity': self._calculate_severity(scores),
            'timestamp': datetime.now()
        }
        
        self.last_prediction_time = results['timestamp']
        return results
    
    def _calculate_severity(self, scores):
        """
        Calculate anomaly severity based on scores
        """
        # Normalize scores to 0-1 range (lower scores = more anomalous)
        min_score = -0.5  # Typical range for isolation forest
        max_score = 0.5
        
        normalized_scores = (scores - min_score) / (max_score - min_score)
        normalized_scores = np.clip(normalized_scores, 0, 1)
        
        # Invert so higher values = more severe
        severity = 1 - normalized_scores
        
        return severity

# Initialize anomaly detection pipeline
print("🔧 Initializing anomaly detection pipeline...")
anomaly_pipeline = AnomalyDetectionPipeline()

# Test the pipeline with sample data
print("\n🧪 Testing anomaly detection pipeline...")
test_features = np.random.normal(0, 1, (5, len(anomaly_pipeline.feature_columns)))
test_results = anomaly_pipeline.predict_anomaly(test_features)

print(f"  Predictions: {test_results['is_anomaly'].sum()} anomalies detected")
print(f"  Severity range: {test_results['severity'].min():.3f} - {test_results['severity'].max():.3f}")
print("✅ Pipeline test successful")

## Self-Healing Workflow Implementation

Implement the complete self-healing workflow from anomaly detection to remediation.

In [ ]:
class SelfHealingWorkflow:
    """
    Complete self-healing workflow implementation
    """
    
    def __init__(self, anomaly_pipeline, coord_client):
        self.anomaly_pipeline = anomaly_pipeline
        self.coord_client = coord_client
        self.remediation_history = []
        self.cooldown_tracker = {}
        
    def process_metrics(self, metrics_data):
        """
        Process incoming metrics and trigger self-healing if needed
        """
        print(f"📊 Processing metrics at {datetime.now()}")
        
        # Convert metrics to feature vector
        features = self._extract_features(metrics_data)
        
        # Detect anomalies
        anomaly_results = self.anomaly_pipeline.predict_anomaly(features)
        
        # Process each anomaly
        for i, is_anomaly in enumerate(anomaly_results['is_anomaly']):
            if is_anomaly:
                severity = anomaly_results['severity'][i]
                score = anomaly_results['anomaly_scores'][i]
                
                print(f"🚨 Anomaly detected! Severity: {severity:.3f}, Score: {score:.3f}")
                
                # Create anomaly record
                anomaly_record = {
                    'timestamp': anomaly_results['timestamp'].isoformat(),
                    'severity': float(severity),
                    'anomaly_score': float(score),
                    'metrics': metrics_data,
                    'source': 'isolation_forest_model'
                }
                
                # Determine and execute remediation
                remediation_action = self._select_remediation_action(severity)
                if remediation_action:
                    self._execute_remediation(anomaly_record, remediation_action)
        
        return anomaly_results
    
    def _extract_features(self, metrics_data):
        """
        Extract features from metrics data for anomaly detection
        """
        # For demo purposes, generate random features
        # In real implementation, this would extract actual metrics
        n_features = len(self.anomaly_pipeline.feature_columns)
        
        # Simulate feature extraction from metrics
        features = np.random.normal(0, 1, n_features)
        
        # Add some realistic patterns
        if 'cpu_high' in str(metrics_data):
            features[0] += 2  # Simulate high CPU feature
        if 'memory_leak' in str(metrics_data):
            features[1] += 3  # Simulate memory issue feature
        
        return features.reshape(1, -1)
    
    def _select_remediation_action(self, severity):
        """
        Select appropriate remediation action based on severity
        """
        # Sort actions by severity threshold (descending)
        sorted_actions = sorted(
            REMEDIATION_ACTIONS.items(),
            key=lambda x: x[1]['severity_threshold'],
            reverse=True
        )
        
        # Find first action that matches severity
        for action_name, action_config in sorted_actions:
            if severity >= action_config['severity_threshold']:
                # Check cooldown
                if self._check_cooldown(action_name, action_config['cooldown_minutes']):
                    return action_name
                else:
                    print(f"⏰ Action {action_name} in cooldown period")
        
        return None
    
    def _check_cooldown(self, action_name, cooldown_minutes):
        """
        Check if action is in cooldown period
        """
        if action_name not in self.cooldown_tracker:
            return True
        
        last_execution = self.cooldown_tracker[action_name]
        cooldown_period = timedelta(minutes=cooldown_minutes)
        
        return datetime.now() - last_execution > cooldown_period
    
    def _execute_remediation(self, anomaly_record, action_name):
        """
        Execute remediation action
        """
        print(f"🔧 Executing remediation action: {action_name}")
        
        # Create remediation request
        remediation_request = {
            'action': action_name,
            'anomaly': anomaly_record,
            'timestamp': datetime.now().isoformat(),
            'description': REMEDIATION_ACTIONS[action_name]['description']
        }
        
        # Submit to coordination engine
        success, response = self.coord_client.submit_anomaly({
            'type': 'remediation_request',
            'data': remediation_request
        })
        
        if success:
            print(f"✅ Remediation request submitted successfully")
            remediation_id = response.get('remediation_id', 'unknown')
        else:
            print(f"❌ Failed to submit remediation request - executing locally")
            remediation_id = f"local_{int(time.time())}"
            self._execute_local_remediation(action_name, anomaly_record)
        
        # Update cooldown tracker
        self.cooldown_tracker[action_name] = datetime.now()
        
        # Record in history
        self.remediation_history.append({
            'remediation_id': remediation_id,
            'action': action_name,
            'timestamp': datetime.now(),
            'severity': anomaly_record['severity'],
            'success': success
        })
    
    def _execute_local_remediation(self, action_name, anomaly_record):
        """
        Execute remediation action locally (simulation)
        """
        print(f"🔧 Simulating local remediation: {action_name}")
        
        if action_name == 'pod_restart':
            print("  🔄 Simulating pod restart...")
            time.sleep(1)  # Simulate action time
            print("  ✅ Pod restart completed")
            
        elif action_name == 'scale_up':
            print("  📈 Simulating deployment scale up...")
            time.sleep(2)  # Simulate action time
            print("  ✅ Scale up completed")
            
        elif action_name == 'resource_adjustment':
            print("  ⚙️ Simulating resource adjustment...")
            time.sleep(1.5)  # Simulate action time
            print("  ✅ Resource adjustment completed")
            
        elif action_name == 'alert_escalation':
            print("  🚨 Simulating alert escalation...")
            print("  📧 Alert sent to operations team")
            print("  ✅ Escalation completed")
    
    def get_remediation_summary(self):
        """
        Get summary of remediation actions
        """
        if not self.remediation_history:
            return "No remediation actions executed yet"
        
        df = pd.DataFrame(self.remediation_history)
        
        summary = {
            'total_actions': len(df),
            'success_rate': df['success'].mean(),
            'actions_by_type': df['action'].value_counts().to_dict(),
            'average_severity': df['severity'].mean(),
            'last_action': df.iloc[-1]['timestamp'] if len(df) > 0 else None
        }
        
        return summary

# Initialize self-healing workflow
print("🔧 Initializing self-healing workflow...")
healing_workflow = SelfHealingWorkflow(anomaly_pipeline, coord_client)
print("✅ Self-healing workflow initialized")

## End-to-End Demonstration

Demonstrate the complete self-healing workflow with simulated scenarios.

In [ ]:
# Simulate different anomaly scenarios
print("🎭 Running Self-Healing Demonstration")
print("=" * 50)

# Scenario 1: Normal metrics (should not trigger)
print("\n📊 Scenario 1: Normal Operations")
normal_metrics = {
    'cpu_usage': 45.2,
    'memory_usage': 62.1,
    'pod_count': 12,
    'response_time': 150
}
result1 = healing_workflow.process_metrics(normal_metrics)
print(f"Result: {result1['is_anomaly'].sum()} anomalies detected")

# Scenario 2: High CPU (moderate anomaly)
print("\n📊 Scenario 2: High CPU Usage")
high_cpu_metrics = {
    'cpu_usage': 95.8,
    'memory_usage': 68.3,
    'pod_count': 12,
    'response_time': 450,
    'cpu_high': True  # Trigger feature
}
result2 = healing_workflow.process_metrics(high_cpu_metrics)
print(f"Result: {result2['is_anomaly'].sum()} anomalies detected")

# Scenario 3: Memory leak (severe anomaly)
print("\n📊 Scenario 3: Memory Leak Detected")
memory_leak_metrics = {
    'cpu_usage': 78.2,
    'memory_usage': 98.7,
    'pod_count': 15,
    'response_time': 2500,
    'memory_leak': True  # Trigger feature
}
result3 = healing_workflow.process_metrics(memory_leak_metrics)
print(f"Result: {result3['is_anomaly'].sum()} anomalies detected")

# Scenario 4: Critical system failure
print("\n📊 Scenario 4: Critical System Failure")
critical_metrics = {
    'cpu_usage': 99.9,
    'memory_usage': 99.2,
    'pod_count': 3,  # Many pods crashed
    'response_time': 10000,
    'cpu_high': True,
    'memory_leak': True
}
result4 = healing_workflow.process_metrics(critical_metrics)
print(f"Result: {result4['is_anomaly'].sum()} anomalies detected")

# Wait a moment to simulate time passing
print("\n⏰ Waiting 2 seconds...")
time.sleep(2)

# Scenario 5: Test cooldown (should not trigger same action)
print("\n📊 Scenario 5: Testing Cooldown Period")
result5 = healing_workflow.process_metrics(critical_metrics)  # Same critical metrics
print(f"Result: {result5['is_anomaly'].sum()} anomalies detected")

print("\n🎉 Demonstration completed!")

In [ ]:
# Display remediation summary
print("📋 Self-Healing Summary")
print("=" * 50)

summary = healing_workflow.get_remediation_summary()

if isinstance(summary, dict):
    print(f"Total remediation actions: {summary['total_actions']}")
    print(f"Success rate: {summary['success_rate']:.1%}")
    print(f"Average severity: {summary['average_severity']:.3f}")
    
    print("\nActions by type:")
    for action, count in summary['actions_by_type'].items():
        description = REMEDIATION_ACTIONS[action]['description']
        print(f"  {action}: {count} times - {description}")
    
    if summary['last_action']:
        print(f"\nLast action: {summary['last_action']}")
else:
    print(summary)

# Display cooldown status
print("\n⏰ Cooldown Status:")
current_time = datetime.now()
for action_name, last_execution in healing_workflow.cooldown_tracker.items():
    cooldown_minutes = REMEDIATION_ACTIONS[action_name]['cooldown_minutes']
    time_since = current_time - last_execution
    remaining = timedelta(minutes=cooldown_minutes) - time_since
    
    if remaining.total_seconds() > 0:
        print(f"  {action_name}: {remaining.total_seconds():.0f}s remaining")
    else:
        print(f"  {action_name}: Ready")

print("\n✅ Self-healing workflow demonstration completed successfully!")
print("\n🔗 Next steps:")
print("  1. Deploy trained models to production")
print("  2. Configure real Prometheus metrics collection")
print("  3. Set up monitoring and alerting for the self-healing system")
print("  4. Test with real OpenShift workloads")